# Durchführung eines Diagnose- und Predicitve Maintenance-Projekts (Lea Sagemüller, Enzo Zacharias)

Im Rahmen des Wahlmoduls Diagnose und Predictive Maintenance im fünften Semester des Studiengangs Digitale Technologien wurde als Modulprüfung eine Hausarbeit verfasst, die ein Projekt im Diagnose- und Predictive Maintenance-Bereich beinhaltet. Aus den behandelten Kapiteln der Lehrveranstaltung sollten jeweils Inhalte aufgegriffen werden und auf einen ausgewählten Datensatz angewendet werden. Der Ablauf des Projektes unterteilt sich in die Datenbeschreibung, die Datenvorverarbeitung, die Anomalieerkennung, Predicitve Maintenance sowie die Diagnose und Fehlerursachenerkennung. 

## Auswahl des Datensatzes

Für die Durchführung des Projektes wurde ein passender Datensatz ausgewählt, an welchem die oben genannten Schritte durchgeführt werden können. Der Microsoft Azure Predictive Maintenance-Datensatz auf Kaggle bietet dafür eine umfassende Grundlage. Er umfasst verschiedene Datenquellen, darunter Sensordaten, Fehlerprotokolle, Wartungsinformationen sowie Maschinendaten. Ziel des Datensatzes ist es, Muster und Zusammenhänge zu erkennen, die dabei helfen, Ausfälle frühzeitig vorherzusagen und die Wartungsplanung zu optimieren. 
Die Daten erhalten zeitlich gestempelte Sensormessungen, die den Zustand und Betrieb von Maschinen überwachen, beispielsweise Temperatur- oder Druckwerte. Ergänzend dazu dokumentieren Fehlerprotokolle aufgetretene Störungen, während Wartungslogs Informationen über geplante oder ungeplante Instandhaltungsmaßnahmen bereitstellen. Stammdaten zu den Maschinen, wie Modelltyp oder Alter, sowie Details zu tatsächlichen Ausfällen, einschließlich Ausfalltyp und Zeitstempel, sind ebenfalls Teil des Datensatzes.

## Einlesen der Datensätze und Zusammenführung

### Import der Haupttabelle

In [ ]:
import pandas as pd
df = pd.read_csv(r'..\data\PdM_telemetry.csv')
df.head()

Zunächst wird die Haupttabelle "PdM_telemetry.csv" importiert, im Folgenden die vier zusätzlichen Tabellen.

### Zusatz-Tabellen importieren

In [ ]:
import pandas as pd
df_errors = pd.read_csv(r'..\data\PdM_errors.csv')
df_failures = pd.read_csv(r'..\data\PdM_failures.csv')
df_maint = pd.read_csv(r'..\data\PdM_maint.csv')
df_machines = pd.read_csv(r'..\data\PdM_machines.csv')

### Ausgabe der einzelnen Datensätze

Nun werden jeweils die ersten fünf Einträge aus den erstellten Dataframes der einzelnen Datensätze ausgegeben.

In [ ]:
# Ausgabe der Errors
df_errors.head()

In [ ]:
# Ausgabe der Failures
df_failures.head()

In [ ]:
# Ausgabe von Maint
df_maint.head()

In [ ]:
# Ausgabe von Machines
df_machines.head()

Hier weiß ich nicht mehr, was genau der Plan war:

Hier könnte man diese Tabelle exemplarisch als Beispiel aufzeigen --> Danach zusammenführen in einem Dataframe in Bezug auf machineID und datetime

In [ ]:
import pandas as pd
import os

# Definieren des Dateipfades
directory = r'../data'
files = [r'PdM_errors.csv', r'PdM_failures.csv', r'PdM_machines.csv', r'PdM_maint.csv', r'PdM_telemetry.csv']

dfs = {}
for file in files:
    filepath = os.path.join(directory, file)
    df_name = file.split('.')[0]
    dfs[df_name] = pd.read_csv(filepath)

Es können nun die einzelnen Dataframes zu einem gemeinsamen Dataframe in Bezug auf die Spalten "datetime" und "machineID" zusammengeführt werden.

In [ ]:
merged_df = dfs['PdM_telemetry']
merged_df = merged_df.merge(dfs['PdM_errors'], on=['datetime', 'machineID'], how='left')
merged_df = merged_df.merge(dfs['PdM_failures'], on=['datetime', 'machineID'], how='left')
merged_df = merged_df.merge(dfs['PdM_maint'], on=['datetime', 'machineID'], how='left')
merged_df = merged_df.merge(dfs['PdM_machines'], on=['machineID'], how='left')

merged_df.head()

# Speichern des zusammengeführten Dataframes
merged_df.to_csv(r'../data/merged_df.csv', index=False)

## Datenbeschreibung

Im Folgenden wird der im CSV-Format vorliegende zusammengeführte Datensatz importiert und in einen Dataframe eingelesen. Ebenfalls in einen Dataframe eingelesen wird der Datensatz, der die Fehlerinformationen enthält. 

In [ ]:
import pandas as pd
df = pd.read_csv(r'../data/PdM_merged.csv')

df_error = pd.read_csv(r'../data/PdM_errors.csv')

### Anschauen der Daten

In [ ]:
df.head()

Es werden die ersten fünf Einträge des eingelesenen Datensatzes angezeigt, um mehr über die Struktur der Daten zu erfahren.

In [ ]:
df.describe()

Die Beschreibung der Daten ist hilfreich, um Informationen wie Minimal- und Maximalwert sowie Mittelwert oder Standardabweichung zu untersuchen und dort ggf. schon erste Auffälligkeiten erkennen zu können.

In [ ]:
df.info()

Es werden nun die einzelnen Spalten mit ihren zugehörigen Datentypen angezeigt. Dabei fällt direkt auf, dass die Spalte "datetime" nicht im richtigen Format vorliegt und im weiteren Verlauf angepasst werden muss.

### Plotten aller Messwerte (pro Tag / pro Monat)

In [ ]:
# Zeige die Spannung (voltage), Drehzahl (rotate), den Druck (pressure) und die Vibration (vibration) in jeweils einem Plot

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter
date_format = DateFormatter("%H:%M")

# Filtern der Daten für die ersten 24 Stunden
df_filtered = df[(df['datetime'] >= '2015-01-01 06:00:00') & (df['datetime'] < '2015-01-02 06:00:00') & (df['machineID'] == 1)]
fig, axs = plt.subplots(4, 1, figsize=(12, 24), sharex=True)

# Plot für Spannung (voltage)
sns.lineplot(x='datetime', y='volt', data=df_filtered, ax=axs[0], color='b')
axs[0].set_title('Voltage per Machine per Day (First 24 Hours)')
axs[0].set_ylabel('Voltage')
axs[0].grid(True)

# Plot für Drehzahl (rotate)
sns.lineplot(x='datetime', y='rotate', data=df_filtered, ax=axs[1], color='g')
axs[1].set_title('Rotate per Machine per Day (First 24 Hours)')
axs[1].set_ylabel('Rotate')
axs[1].grid(True)

# Plot für Druck (pressure)
sns.lineplot(x='datetime', y='pressure', data=df_filtered, ax=axs[2], color='r')
axs[2].set_title('Pressure per Machine per Day (First 24 Hours)')
axs[2].set_ylabel('Pressure')
axs[2].grid(True)

# Plot für Vibration (vibration) 
sns.lineplot(x='datetime', y='vibration', data=df_filtered, ax=axs[3], color='m')
axs[3].set_title('Vibration per Machine per Day (First 24 Hours)')
axs[3].set_ylabel('Vibration')
axs[3].set_xlabel('Datetime')
axs[3].grid(True)

# Formatieren der x-Achse, um nur die Stunden anzuzeigen
for ax in axs:
    ax.xaxis.set_major_formatter(date_format)

plt.tight_layout()
plt.show()

In diesem Code werden für die erste Maschine ("machineID" = 1) die Messdaten für Spannung (Voltage), Drehzahl (Rotate), Druck (Pressure) und Vibration in separaten Diagrammen visualisiert. Die Daten werden auf einen Zeitraum von 24 Stunden (vom 1. Januar 2015, 06:00 Uhr bis zum 2. Januar 2015, 06:00 Uhr) gefiltert. Dabei zeigt die x-Achse nur den Zeitstempel, formatiert als Uhrzeit (Stunden:Minuten), da nur auf 24 Stunden gefiltert wurde. 

In [ ]:
# Zeige die Spannung (voltage), Drehzahl (rotate), den Druck (pressure) und die Vibration (vibration) in jeweils einem Plot

from matplotlib.dates import DateFormatter
date_format = DateFormatter("%d-%m-%Y")

# Filtern der Daten für die ersten 31 Tage
df_filtered = df[(df['datetime'] >= '2015-01-01 06:00:00') & (df['datetime'] < '2015-01-31 23:59:00') & (df['machineID'] == 1)]
fig, axs = plt.subplots(4, 1, figsize=(12, 24), sharex=True)

# Plot für Spannung (voltage)
sns.lineplot(x='datetime', y='volt', data=df_filtered, ax=axs[0], color='b')
axs[0].set_title('Voltage per Machine per Day (First 31 Days)')
axs[0].set_ylabel('Voltage')
axs[0].grid(True)

# Plot für Drehzahl (rotate)
sns.lineplot(x='datetime', y='rotate', data=df_filtered, ax=axs[1], color='g')
axs[1].set_title('Rotate per Machine per Day (First 31 Days)')
axs[1].set_ylabel('Rotate')
axs[1].grid(True)

# Plot für Druck (pressure)
sns.lineplot(x='datetime', y='pressure', data=df_filtered, ax=axs[2], color='r')
axs[2].set_title('Pressure per Machine per Day (First 31 Days)')
axs[2].set_ylabel('Pressure')
axs[2].grid(True)

# Plot für Vibration (vibration)
sns.lineplot(x='datetime', y='vibration', data=df_filtered, ax=axs[3], color='m')
axs[3].set_title('Vibration per Machine per Day (First 31 Days)')
axs[3].set_ylabel('Vibration')
axs[3].set_xlabel('Datetime')
axs[3].grid(True)

# Formatieren der x-Achse, um nur die Tage anzuzeigen 
for ax in axs:
    ax.xaxis.set_major_formatter(date_format)

plt.tight_layout()
plt.show()

Dieser Code ist fast identisch zum vorherigen mit dem Unterschied, dass nun ein Zeitraum von 31 Tagen betrachtet wird und demnach auch die x-Achse in Tagen statt in Stunden und Minuten angezeit werden soll.

### Anzeigen der größten und kleinsten Ausprägungen pro Attribut

In [ ]:
min_timestamp = df['datetime'].min()
max_timestamp = df['datetime'].max()

print(f'Minimum timestamp: {min_timestamp}')
print(f'Maximum timestamp: {max_timestamp}\n')

min_volt = df['volt'].min()
max_volt = df['volt'].max()

print(f'Minimum voltage: {min_volt:.2f}')
print(f'Maximum voltage: {max_volt:.2f}\n')

min_rotate = df['rotate'].min()
max_rotate = df['rotate'].max()

print(f'Minimum rotate: {min_rotate:.2f}')
print(f'Maximum rotate: {max_rotate:.2f}\n')

min_pressure = df['pressure'].min()
max_pressure = df['pressure'].max()

print(f'Minimum pressure: {min_pressure:.2f}')
print(f'Maximum pressure: {max_pressure:.2f}\n')

min_vibration = df['vibration'].min()
max_vibration = df['vibration'].max()

print(f'Minimum vibration: {min_vibration:.2f}')
print(f'Maximum vibration: {max_vibration:.2f}\n')

min_age = df['age'].min()
max_age = df['age'].max()

print(f'Minimum age: {min_age:.2f}')
print(f'Maximum age: {max_age:.2f}')

Es werden nun die minimalen und maximalen Werte für verschiedene Attribute im Dataframe berechnet und ausgegeben.Zunächst wird der früheste und späteste Zeitstempel der Datensätze ermittelt und angezeigt. Danach werden die minimalen und maximalen Werte der numerischen Variablen volt (Spannung), rotate (Drehzahl), pressure (Druck), vibration (Vibration) sowie age (Alter der Maschine) berechnet. Diese Analyse gibt einen Überblick über die Verteilung und die Spannweite der Daten.

### Untersuchung der Korrelation

In [ ]:
# Formal werden alle Strings von errorID, failure und comp in int-Werte umgewandelt (beachtet werden die NaN-Werte dabei als 0, sonst als 1)
db_bfr = df.copy()
db_bfr['errorID'] = db_bfr['errorID'].apply(lambda x: 0 if pd.isnull(x) else 1)
db_bfr['failure'] = db_bfr['failure'].apply(lambda x: 0 if pd.isnull(x) else 1)
db_bfr['comp'] = db_bfr['comp'].apply(lambda x: 0 if pd.isnull(x) else 1)
correlation = db_bfr[['volt', 'rotate', 'pressure', 'vibration', 'age', 'errorID', 'failure', 'comp']].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

Zur Untersuchung der Korrelation werden zunächst die Spalten errorID, failure und comp in numerische Werte umgewandelt, wobei NaN-Werte als 0 und alle anderen als 1 interpretiert werden. Anschließend wird eine Korrelationsmatrix berechnet, die die linearen Zusammenhänge zwischen den Variablen volt, rotate, pressure, vibration, age, errorID, failure und comp darstellt. Diese wird mit einer Heatmap visualisiert. Die Darstellung ermöglicht es, potenzielle Zusammenhänge und Einflussfaktoren auf Fehler und Ausfälle zu erkennen. Dabei ist in diesem Fall eine Korrelation zwischen failure und comp erkennbar.

In [ ]:
# Autokorrelation von volt, rotate, pressure und vibration (Korrelation mit sich selbst in Bezug auf die Zeit)

from statsmodels.graphics.tsaplots import plot_acf

fig, axs = plt.subplots(4, 1, figsize=(12, 24))

# Autokorrelation von volt
plot_acf(df['volt'], lags=100, ax=axs[0])
axs[0].set_title('Autocorrelation of Voltage')

# Autokorrelation von rotate
plot_acf(df['rotate'], lags=100, ax=axs[1])
axs[1].set_title('Autocorrelation of Rotate')

# Autokorrelation von pressure
plot_acf(df['pressure'], lags=100, ax=axs[2])
axs[2].set_title('Autocorrelation of Pressure')

# Autokorrelation von vibration
plot_acf(df['vibration'], lags=100, ax=axs[3])
axs[3].set_title('Autocorrelation of Vibration')

plt.tight_layout()
plt.show()


Anschließend wird die Autokorrelation für die Variablen volt (Spannung), rotate (Drehzahl), pressure (Druck) und vibration (Vibration) berechnet und in separaten Plots dargestellt. Autokorrelation beschreibt, wie stark eine Zeitreihe mit sich selbst in unterschiedlichen Zeitabständen (Lags) korreliert.

Jeder Plot zeigt die Autokorrelation für bis zu 100 Lags, also Zeitpunkte, die jeweils um eine bestimmte Anzahl von Intervallen voneinander entfernt sind. Ein Wert nahe 1 deutet auf eine starke positive Korrelation hin, ein Wert nahe -1 auf eine starke negative Korrelation. Werte nahe 0 zeigen keine Korrelation. Die Plots ermöglichen es, Muster oder saisonale Abhängigkeiten in den Zeitreihen zu identifizieren.

In allen vier Fällen ist eine schwache Korrelation von höchstens 0,2 zu erkennen.







### Ausfälle pro Maschine

In [ ]:
import numpy as np

# Anzahl Ausfälle pro Maschine und errorID als DataFrame erstellen und sortieren
failure_count = df_error.groupby(['machineID', 'errorID']).size().unstack(fill_value=0)
failure_count_sorted = failure_count.sum(axis=1).sort_values(ascending=False)

# Figur mit angepasster Größe erstellen
plt.figure(figsize=(15, 10))  

# Balkendiagramm erstellen
bottom = np.zeros(len(failure_count_sorted))
colors = ['#2E86C1', '#E74C3C', '#F1C40F', '#2ECC71', '#9B59B6']  

for i, errorID in enumerate(failure_count.columns):
    values = failure_count.loc[failure_count_sorted.index, errorID]
    plt.bar(range(len(failure_count_sorted)), values, bottom=bottom, color=colors[i % len(colors)], label=errorID)
    bottom += values

# X-Achsen-Labels mit den entsprechenden Maschinen-IDs
plt.xticks(range(len(failure_count_sorted)), failure_count_sorted.index, 
           rotation=90,  
           ha='center',  
           fontsize=8)   

plt.title('Fehler pro Maschine', fontsize=14, pad=20)
plt.xlabel('Maschinen ID', fontsize=12, labelpad=10)  
plt.ylabel('Anzahl Fehler', fontsize=12)

# Gitter hinzufügen
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Ränder anpassen
plt.margins(x=0.01)

plt.subplots_adjust(bottom=0.2)  

# Legende hinzufügen
plt.legend(title='Error ID')

plt.show()

Um die Ausfälle pro Maschine untersuchen zu können, wird ein gestapeltes Balkendiagramm erstellt, das die Anzahl der Fehler pro Maschine und Fehler-ID darstellt. Zunächst werden die Fehlerdaten nach machineID und errorID gruppiert und die Anzahl der Fehler berechnet, wobei fehlende Werte mit 0 aufgefüllt werden. Anschließend werden die Maschinen nach der Gesamtanzahl ihrer Fehler sortiert. Im Diagramm repräsentiert jeder Balken die Gesamtanzahl der Fehler pro Maschine. Das Diagramm ermöglicht es, die fehleranfälligsten Maschinen sowie die Verteilung der Fehlerarten zu erkennen.

### Anzahl an Fehlerklassen

In [ ]:
plt.figure(figsize=(10, 5))

# Balkendiagramm erstellen mit x = Error-Klasse und y = Anzahl der Fehler
sns.countplot(x='errorID', data=df_error, order=df_error['errorID'].value_counts().index, palette=colors)

plt.title('Anzahl der Fehler', fontsize=14, pad=20)
plt.ylabel('Anzahl der Fehler', fontsize=12)
plt.xlabel('Error ID', fontsize=12)

plt.margins(x=0.01)
plt.subplots_adjust(bottom=0.2)
plt.show()


Mittels eines Balkendiagramms wird die Verteilung der Fehlerarten visualisiert. Dabei wird auf der x-Achse die errorID (Fehlerklasse) und auf der y-Achse die Anzahl der Fehler dargestellt. Die Fehlerarten werden nach ihrer Häufigkeit sortiert. 

### Anzahl der Ausfälle

In [ ]:
plt.figure(figsize=(10, 5))

# Balkendiagramm erstellen mit x = Error-Klasse und y = Anzahl der Fehler
sns.countplot(x='errorID', data=df_error, order=df_error['errorID'].value_counts().index, palette=colors)

plt.title('Anzahl der Fehler', fontsize=14, pad=20)
plt.ylabel('Anzahl der Fehler', fontsize=12)
plt.xlabel('Error ID', fontsize=12)

plt.margins(x=0.01)
plt.subplots_adjust(bottom=0.2)
plt.show()


Ebenso wie bei zuvor dargestellten Anzahl der Fehler können auch die Anzahl der Ausfälle visualisiert werden. Auch hier lässt sich im erstellten Diagramm erkennen, welche Art der Ausfälle wie häufig vorkommt.

### Verteilung des Alters der Maschinen

In [ ]:
from scipy.stats import gaussian_kde
import numpy as np

# Verteilung des Alters der Maschinen in Prozent
age_distribution = df['age'].value_counts(normalize=True) * 100

# Alter 13 mit 0% hinzufügen
age_distribution = age_distribution.reindex(range(21), fill_value=0)

plt.figure(figsize=(10, 5))

# Balkendiagramm erstellen mit x = Alter und y = Prozent
sns.barplot(x=age_distribution.index, y=age_distribution.values, color='b', alpha=0.6)

# Density Kurve hinzufügen
age_values = df['age'].values
kde = gaussian_kde(age_values)
age_range = np.linspace(age_values.min(), age_values.max(), 100)
age_density = kde(age_range) * 100  # Skalieren auf Prozent

plt.plot(age_range, age_density, color='r')

plt.title('Verteilung des Alters der Maschinen', fontsize=14, pad=20)
plt.ylabel('Prozent', fontsize=12)
plt.xlabel('Alter', fontsize=12)

plt.margins(x=0.01)
plt.subplots_adjust(bottom=0.2)
plt.show()

Anhand des erstellten Säulendiagramms lässt sich die Verteilung des Alters der untersuchten Maschinen darstellen. Dabei wird auf der x-Achse das mögliche Alter abgebildet und auf der y-Achse die Verteilung in Prozent. Die meisten Maschinen sind dabei zehn oder 14 Jahre alt.

### Linearer Zusammenhang (Regression) mit der Anzahl an Ausfällen und dem Alter der Maschine

In [ ]:
# Zunächst Summierung der Anzahl an Ausfälle pro Alter
age_error_count = df.groupby('age').size()

# Erstellung einer linearen Darstellung mit Trend (x-Achse = Alter und y = Anzahl an Ausfälle)
plt.figure(figsize=(10, 5))

sns.regplot(x=age_error_count.index, y=age_error_count.values, color='b', scatter_kws={'s': 10})

plt.title('Linearer Zusammenhang zwischen Alter und Anzahl an Ausfällen', fontsize=14, pad=20)
plt.ylabel('Anzahl an Ausfällen', fontsize=12)
plt.xlabel('Alter', fontsize=12)

# Setzen der x-Achse, um jedes Jahr als Strich darzustellen
plt.xticks(ticks=range(age_error_count.index.min(), age_error_count.index.max() + 1))

plt.margins(x=0.01)
plt.subplots_adjust(bottom=0.2)
plt.show()

# Berechnen der Regressionsgeraden
X = age_error_count.index.values.reshape(-1, 1)
y = age_error_count.values

# Erstellung des Modell und Training
model = LinearRegression()
model.fit(X, y)

# Ausgabe der Funktion der Geraden
m = model.coef_[0]
b = model.intercept_
print(f"Die Funktion der Geraden lautet: y = {m}x + {b}")

Im Bereich der Datenbeschreibung wird abschließend noch der Zusammenhang zwischen dem Alter der Maschinen und der Anzahl an Ausfällen visualisiert. Zuerst wird die Anzahl der Ausfälle für jedes Alter summiert. Anschließend wird ein Streudiagramm mit einer linearen Regressionsgeraden erstellt, um den Trend zwischen Alter und Ausfällen darzustellen. Die x-Achse zeigt das Alter der Maschinen und die y-Achse die Anzahl der Ausfälle. Ein lineares Regressionsmodell wird trainiert, um die beste Geradengleichung zu berechnen, die den Zusammenhang beschreibt. Die Koeffizienten der Regressionsgeraden werden schließlich ausgegeben.

## Datenvorberarbeitung


### Umwandlung einzelner Spalten in integer-Werte

In [ ]:
# errorID, failure und comp sind nicht numerisch und müssen umgewandelt werden
# Wenn NaN, dann wird 0 eingesetzt, sonst wenn errorID = error1 dann 1, errorID = error2 dann 2, errorID = error3 dann 3,....

df['errorID'] = df['errorID'].fillna(0)
df['errorID'] = df['errorID'].replace('error1', 1)
df['errorID'] = df['errorID'].replace('error2', 2)
df['errorID'] = df['errorID'].replace('error3', 3)
df['errorID'] = df['errorID'].replace('error4', 4)
df['errorID'] = df['errorID'].replace('error5', 5)

df['failure'] = df['failure'].fillna(0)
df['failure'] = df['failure'].replace('comp1', 1)
df['failure'] = df['failure'].replace('comp2', 2)
df['failure'] = df['failure'].replace('comp3', 3)
df['failure'] = df['failure'].replace('comp4', 4)

df['comp'] = df['comp'].fillna(0)
df['comp'] = df['comp'].replace('comp1', 1)
df['comp'] = df['comp'].replace('comp2', 2)
df['comp'] = df['comp'].replace('comp3', 3)
df['comp'] = df['comp'].replace('comp4', 4)

df['model'] = df['model'].fillna(0)
df['model'] = df['model'].replace('model1', 1)
df['model'] = df['model'].replace('model2', 2)
df['model'] = df['model'].replace('model3', 3)
df['model'] = df['model'].replace('model4', 4)

Im ersten Schritt der Datenvorverarbeitung werden die Spalten errorID, failure, comp und model umgewandelt, damit sie nur noch numerische Werte enthalten. Zuerst werden alle NaN-Werte in diesen Spalten durch 0 ersetzt. Danach werden die einzelnen Kategorien, wie beispielweise error1, error2, comp1 oder comp2 durch numerische Werte ersetzt: error1 wird zu 1, error2 zu 2 etc. Dies wird für alle genannten Spalten durchgeführt. Dadurch liegen alle Werte in diesen Spalten als numerische Werte vor und können so für die weitere Analyse verwendet werden.

### Missing Value Treatment

In [ ]:
# Testen auf fehlende Werte
print(df.isnull().sum())

Beim Missing Value Treatment wird der Dataframe auf fehlende Werte überprüft. Dies ist hier jedoch nicht der Fall, da für alle Variablen der Wert 0 ausgegeben wird. Dadurch ist ebenfalls keine Interpolation oder ähnliches notwendig.

### Ausreißer-Analyse (statistisch und dichtebasiert)

#### Data Cleaning (Ausreißer identifizieren und entfernen)

In [ ]:
# Gaußsche Normalverteilung der Attribute (volt, rotate, pressure, vibration)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="whitegrid")
plt.figure(figsize=(20, 10))

# Volt
plt.subplot(2, 2, 1)
sns.histplot(df['volt'])
plt.title(f'Volt (Skewness: {df["volt"].skew():.2f})')

# Rotate
plt.subplot(2, 2, 2)
sns.histplot(df['rotate'])
plt.title(f'Rotate (Skewness: {df["rotate"].skew():.2f})')

# Pressure
plt.subplot(2, 2, 3)
sns.histplot(df['pressure'])
plt.title(f'Pressure (Skewness: {df["pressure"].skew():.2f})')

# Vibration
plt.subplot(2, 2, 4)
sns.histplot(df['vibration'])
plt.title(f'Vibration (Skewness: {df["vibration"].skew():.2f})')

plt.show()


Für das Data Cleaning werden die Verteilungen der vier Attribute volt, rate, pressure und vibration mithilfe von Histogrammen visualisiert. Für jedes Attribut wird zusätzlich die Schiefe berechnet, die im Titel des jeweiligen Plots angezeigt wird. Die Schiefe gibt an, wie asymmetrisch die Verteilung der Daten ist. Ein Wert von 0 deutet auf eine symmetrische Verteilung hin, während positive oder negative Werte auf eine Rechts- bzw. Linksschiefe hinweisen. Diese Visualisierung hilft dabei, die Verteilungen der Attribute besser zu verstehen und zu erkennen, ob sie einer Normalverteilung entsprechen oder signifikante Verzerrungen aufweisen.

In [ ]:
# Statistische Ausreißer-Analyse (Mittelwert, Median und Standardabweichung)

# Mittelwert

print("Volt-Mittelwert: ",df['volt'].mean())
print("Rotation-Mittelwert: ",df['rotate'].mean())
print("Pressure-Mittelwert: ",df['pressure'].mean())
print("Vibration-Mittelwert: ",df['vibration'].mean())

# Median

print("Volt-Median: ",df['volt'].median())
print("Rotation-Median: ",df['rotate'].median())
print("Pressure-Median: ",df['pressure'].median())
print("Vibration-Median: ",df['vibration'].median())

# Standardabweichung

print("Volt-Standardabweichung: ",df['volt'].std())
print("Rotation-Standardabweichung: ",df['rotate'].std())
print("Pressure-Standardabweichung: ",df['pressure'].std())
print("Vibration-Standardabweichung: ",df['vibration'].std())



In diesem Code wird eine statistische Ausreißer-Analyse für die vier Attribute volt, rotate, pressure und vibration durchgeführt. Es werden der Mittelwert, der Median und die Standardabweichung als zentrale Kennwerte berechnet. Der Mittelwert gibt den Durchschnitt der Werte an und kann durch extreme Ausreißer verzerrt werden. Der Median stellt den zentralen Wert der Verteilung dar und ist weniger empfindlich gegenüber Ausreißern. Die Standardabweichung misst die Streuung der Werte um den Mittelwert und hilft, die Variabilität der Daten zu verstehen. Diese Kennzahlen bieten eine Grundlage, um mögliche Ausreißer zu identifizieren, die sich signifikant vom Mittelwert oder Median unterscheiden oder eine hohe Streuung aufweisen.

In [ ]:
# Ausreißer-Analyse mit Boxplot

import matplotlib.pyplot as plt
import seaborn as sns

sns.boxplot(x=df['volt'])
plt.show()

# Anzahl der Ausreißer
Q1_volt = df['volt'].quantile(0.25)
Q3_volt = df['volt'].quantile(0.75)

IQR_volt = Q3_volt - Q1_volt

print("Anzahl der Ausreißer: ", ((df['volt'] < (Q1_volt - 1.5 * IQR_volt)) | (df['volt'] > (Q3_volt + 1.5 * IQR_volt))).sum())

# Ausreißer-Analyse für rotate
sns.boxplot(x=df['rotate'])
plt.show()

Q1_rotate = df['rotate'].quantile(0.25)
Q3_rotate = df['rotate'].quantile(0.75)
IQR_rotate = Q3_rotate - Q1_rotate

print("Anzahl der Ausreißer (rotate): ", ((df['rotate'] < (Q1_rotate - 1.5 * IQR_rotate)) | (df['rotate'] > (Q3_rotate + 1.5 * IQR_rotate))).sum())

# Ausreißer-Analyse für pressure
sns.boxplot(x=df['pressure'])
plt.show()

Q1_pressure = df['pressure'].quantile(0.25)
Q3_pressure = df['pressure'].quantile(0.75)
IQR_pressure = Q3_pressure - Q1_pressure

print("Anzahl der Ausreißer (pressure): ", ((df['pressure'] < (Q1_pressure - 1.5 * IQR_pressure)) | (df['pressure'] > (Q3_pressure + 1.5 * IQR_pressure))).sum())

# Ausreißer-Analyse für vibration
sns.boxplot(x=df['vibration'])
plt.show()

Q1_vibration = df['vibration'].quantile(0.25)
Q3_vibration = df['vibration'].quantile(0.75)
IQR_vibration = Q3_vibration - Q1_vibration

print("Anzahl der Ausreißer (vibration): ", ((df['vibration'] < (Q1_vibration - 1.5 * IQR_vibration)) | (df['vibration'] > (Q3_vibration + 1.5 * IQR_vibration))).sum())

Ebenfalls durchgeführt wird eine Ausreißer-Analyse für die Attribute volt, rotate, pressure und vibration. Auch hier werden Boxplots erstellt, um visuell Ausreißer zu identifizieren. Anschließend wird der Interquartilsabstand (IQR) berechnet, und Ausreißer werden als Werte außerhalb des Bereichs Q1 - 1.5 * IQR und Q3 + 1.5 * IQR bestimmt. Die Anzahl der Ausreißer für jedes Attribut wird gezählt und angezeigt. Es ergibt sich eine Anzahl von 7480 Ausreißern in der gesamten Analyse und insgesamt die meisten Ausreißer für das Attribut pressure.

### Datennormalisierung


In [ ]:
# Daten normalisieren auf Werte zwischen 0 und 1

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[['volt', 'rotate', 'pressure', 'vibration', 'age']] = scaler.fit_transform(df[['volt', 'rotate', 'pressure', 'vibration', 'age']])

df.head()

Mithilfe des MinMaxScalers werden die Werte der Attribute volt, rotate, pressure, vibration und age normalisiert. Der MinMaxScaler skalsiert die Werte jedes Attributs auf einen Bereich zwischen 0 und 1. Die normalisierten Werte werden dann in die ursprüngliche DataFrame übernommen und das Ergebnis wird durch df.head() angezeigt.

### Datentransformation

In [ ]:
# Gaußsche Normalverteilung der Attribute (volt, rotate, pressure, vibration)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="whitegrid")
plt.figure(figsize=(10, 20))

# Volt
plt.subplot(4, 1, 1)
sns.histplot(df['volt'])

# Rotate
plt.subplot(4, 1, 2)
sns.histplot(df['rotate'])

# Pressure
plt.subplot(4, 1, 3)
sns.histplot(df['pressure'])

# Vibration
plt.subplot(4, 1, 4)
sns.histplot(df['vibration'])

plt.tight_layout()
plt.show()

Nach der Datennormalisierung besteht nun auch die Möglichkeit, die Verteilungen der Attribute visuell darzustellen. Dafür werden wieder Histrogramme verwendet, die die normalisierten Daten für die einzelnen Attribute darstellen. Dieser Prozess hilft dabei, die Verteilung der Daten zu überprüfen und mögliche Abweichungen von einer normalen (Gaußschen) Verteilung zu erkennen.

### Feature Engineering

In [ ]:
# Feature Engineering (Volt und Rotate / Pressure und Vibration)

# Volt und Rotate
df['volt_rotate'] = df['volt'] * df['rotate']

# Pressure und Vibration
df['pressure_vibration'] = df['pressure'] * df['vibration']

df.head()

Im Rahmen der Datenvorverarbeitung wird nun Feature Engineering durchgeführt, um neue Merkmake (Features) zu erstellen, die durch Kombination bestehender Attribute entstehen. Volt und Rotate bilden das Attribut volt_rotate, Pressure und Vibration das Attribut pressure_vibration. Diese neuen Features könnten nützlich sein, um komplexe Zusammenhänge zwischen den bestehenden Variablen zu erfassen. 

### Feature Importance (der ersten 100.000 Messwerte)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

X = df[['volt', 'rotate', 'pressure', 'vibration', 'age', 'volt_rotate', 'pressure_vibration']].head(100000)
y = df['failure'].head(100000)

rf_model = RandomForestClassifier()
rf_model.fit(X, y)

rf_feat_importances = pd.Series(rf_model.feature_importances_, index=X.columns)
rf_feat_importances.nlargest(10).plot(kind='barh')
plt.show()

Als Abschluss im Rahmen der Datenvorverarbeitung wird ein Random Forest Classifier eingesetzt, um die Bedeutung der verschiedenen Merkmale im Hinblick auf die Vorhersage der Zielgröße failure zu bestimmen. Dazu werden die Eingabedaten aus den Attributen volt, rotate, pressure, vibration, age sowie zwei neu berechneten Merkmalen, volt_rotate und pressure_vibration, zusammengestellt. Die Zielvariable ist failure.

Der Random Forest wird anschließend mit den ersten 100.000 Datenpunkten trainiert. Nach dem Modelltraining werden die Feature Importances extrahiert, welche die relative Bedeutung jedes Merkmals für die Modellvorhersage darstellen. 

### Speicherung

In [ ]:
df.to_pickle("../data/PdM_merged_preprocessed.pkl")

Die vorverarbeiteten Daten werden als pickle-Datei abgespeichert und für die weiteren Analysen verwendet.

## Anomalieerkennung

### DBSCAN-Algorithmus